In [6]:
import pandas as pd
import os
import re
from sklearn.metrics import f1_score

def process_annotation_csvs(folder_path):
    new_dfs = {}
    for filename in os.listdir(folder_path):
        if filename.endswith("_AG.csv"):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, sep='\t')
            new_df = fill_missing_annotation(df)
            df_name = re.search('\d+', filename).group()
            new_dfs[df_name] = new_df
    return new_dfs

def fill_missing_annotation(df):
    for i in range(len(df)):
        if pd.isna(df.iloc[i].annotation):
            df.iloc[i].annotation = df.iloc[i].pos
    return df

In [8]:
df_dict = process_annotation_csvs(os.getcwd())
df_accs = []
df_names = []
df_f1s = []
for key in df_dict.keys():
    current_df = df_dict[key]
    current_df['correct'] = (current_df.annotation == current_df.pos).astype(int)
    current_acc = sum(current_df.correct)/len(current_df)
    current_f1 = f1_score(current_df.annotation, current_df.pos, average='weighted')
    df_f1s.append(current_f1)
    df_names.append(key)
    df_accs.append(current_acc)

In [9]:
df_acc = pd.DataFrame(columns=['epitome', 'annot_acc', 'annot_f1'])
df_acc.epitome = df_names
df_acc.annot_acc = df_accs
df_acc.annot_f1 = df_f1s

In [10]:
df_acc.sort_values(by='epitome')

,epitome,annot_acc,annot_f1
0,0190,0.947977,0.941867
1,0286,0.919192,0.914346
2,0432,0.957746,0.955509
3,0584,0.977612,0.974656
4,0915,0.960894,0.962744
5,1406,0.954455,0.949060
